# Importing necessary libraries

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras import backend as K
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.models import load_model
import numpy as np
import pandas as pd
import glob
import cv2
import csv
import os 
from os import walk

/Users/leannwoo/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Define two functions

getFish: takes in a filepath and reads image data using OpenCV. Resizes image to 150x150 in RGB. Returns the image pixel data, the classifications (eyespot, no eyespot), and the species name. Classification and species name are parsed from the image filepath.

load_kfold_data: takes in k number of folds and a list of image filespaths. Calls getFish to receive image data and appends image data and classification data to respective lists. Label data is converted to binary in order to run in function StratifiedKFold. Convert labels back to categorical. Returns folds, training data, and labels.

In [6]:
def getFish(filepath):
    img = cv2.imread(filepath)
    # standardize size for prediction step
    img = cv2.resize(img, (150, 150))
    img = img/255.0
    img = img.reshape((1,) + img.shape)
    
    if "noeyespot" in filepath:
        classification = "noeyespot"
    else:
        classification = "eyespot"  
    
    _, _, species = filepath.partition('eyespot/')
    
    #res = model.predict(img)
    return(img, classification, species)

def load_data_kfold(k, image_filepaths):
    
    #create lists for image training data and label training data
    x_train = []
    y_train = []
    
    #iterate through list of image filepaths
    #get pixel, label data and store in x,y
    #append x and y to x_train and y_train lists
    #convert classification to binary
    for i in range(len(image_filepaths)):
        x, y, _ = getFish(image_filepaths[i])
        x_train.append(x)
        y_train.append(y)
        if (y_train[i] == 'eyespot'):
            y_train[i] = 0
        else:
            y_train[i] = 1
    
    #StratifiedKFold can only work in binary; convert labels to categorical afterwards
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(x_train, y_train))
    
    #convert labels back to categorical from binary
    y_train = to_categorical(y_train)
    
    return folds, x_train, y_train

# Running load_data_kfold

Load list of image filepaths into "fish_paths" by changing corresponding Dropbox paths  
Use kfold function to get 10 folds, the training data, and the trianing labels.

In [3]:
##CHANGE THESE ACCORDING TO YOUR PATH!!
eye_path = "/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Eyespots_all_species/eyespot/"
noeye_path = "/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Eyespots_all_species/noeyespot/"

fish_paths = []

for (dirpath, dirnames, filenames) in walk(eye_path):
    for i in range(len(filenames)):
        eye_img = dirpath + filenames[i]
        if "Store" not in eye_img:  
            fish_paths.append(eye_img)
    break

for (dirpath, dirnames, filenames) in walk(noeye_path):
    for i in range(len(filenames)):
        noeye_img = dirpath + filenames[i]
        if "Store" not in noeye_img:
            fish_paths.append(noeye_img)
    break

k = 10
folds, x_train, y_train = load_data_kfold(k, fish_paths)

# Defining CNN

In [4]:
# dimensions of our images.
img_width, img_height = 150, 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Creating distorted images generator to create more training images

In [5]:
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 10
                        )

# Running training set using 10-fold cross validation

Now using the training data to train the model that was previously defined with a batch size of 20 and 30 epochs per fold  
  
Training and validation indices come from folds, which was previously defined when we ran load_data_kfold

In [6]:
batch_size = 40

for j, (train_idx, val_idx) in enumerate(folds):
    
    #Creating lists for training and validation batch sets
    X_train_cv = []
    y_train_cv = []
    X_valid_cv = []
    y_valid_cv = []
    
    for i in range(len(train_idx)):
        X_train_cv.append(x_train[train_idx[i]])
        y_train_cv.append(y_train[train_idx[i]])
    for i in range(len(val_idx)):
        X_valid_cv.append(x_train[val_idx[i]])
        y_valid_cv.append(y_train[val_idx[i]])
    
    X_train_cv = np.array(X_train_cv)
    X_train_cv = np.squeeze(X_train_cv, axis = 1)
    
    X_valid_cv = np.array(X_valid_cv)
    X_valid_cv = np.squeeze(X_valid_cv, axis = 1)
    
    y_train_cv = np.array(y_train_cv)
    y_valid_cv = np.array(y_valid_cv)
    
    print '\nFold ',j
    name_weights = "final_model_fold" + str(j) + "_weights.h5"
    generator = gen.flow(X_train_cv, y_train_cv, batch_size = batch_size)
    model.fit_generator(
                generator,
                steps_per_epoch=len(X_train_cv)/batch_size,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data = (X_valid_cv, y_valid_cv))
    model.save_weights('first_try.h5')
    model.save('second_try.h5')
    print(model.evaluate(X_valid_cv, y_valid_cv))


Fold  0
Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.2503 - acc: 0.4500 - val_loss: 0.3065 - val_acc: 0.5000
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 0.3371 - acc: 0.5000 - val_loss: 0.2582 - val_acc: 0.5000
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 0.2824 - acc: 0.5312 - val_loss: 0.2496 - val_acc: 0.5000
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 0.2509 - acc: 0.6000 - val_loss: 0.2514 - val_acc: 0.5000
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.2558 - acc: 0.4688 - val_loss: 0.2490 - val_acc: 0.5000
Epoch 6/100
1/1 [==============================] - 2s 2s/step - loss: 0.2469 - acc: 0.5500 - val_loss: 0.2554 - val_acc: 0.5000
Epoch 7/100
1/1 [==============================] - 2s 2s/step - loss: 0.2661 - acc: 0.4375 - val_loss: 0.2493 - val_acc: 0.6250
Epoch 8/100
1/1 [==============================] - 2s 2s/step - loss: 0.2467 - acc: 0.6500 - va

Epoch 65/100
1/1 [==============================] - 1s 1s/step - loss: 0.2230 - acc: 0.5312 - val_loss: 0.2455 - val_acc: 0.7500
Epoch 66/100
1/1 [==============================] - 2s 2s/step - loss: 0.2156 - acc: 0.6000 - val_loss: 0.2574 - val_acc: 0.5000
Epoch 67/100
1/1 [==============================] - 2s 2s/step - loss: 0.2104 - acc: 0.6875 - val_loss: 0.2434 - val_acc: 0.6250
Epoch 68/100
1/1 [==============================] - 2s 2s/step - loss: 0.2459 - acc: 0.5000 - val_loss: 0.2188 - val_acc: 0.7500
Epoch 69/100
1/1 [==============================] - 2s 2s/step - loss: 0.2428 - acc: 0.5000 - val_loss: 0.2360 - val_acc: 0.6250
Epoch 70/100
1/1 [==============================] - 2s 2s/step - loss: 0.2039 - acc: 0.7000 - val_loss: 0.2399 - val_acc: 0.8750
Epoch 71/100
1/1 [==============================] - 2s 2s/step - loss: 0.2202 - acc: 0.6250 - val_loss: 0.2251 - val_acc: 0.8750
Epoch 72/100
1/1 [==============================] - 2s 2s/step - loss: 0.2141 - acc: 0.6875 - val

Epoch 28/100
1/1 [==============================] - 2s 2s/step - loss: 0.1586 - acc: 0.8750 - val_loss: 0.2601 - val_acc: 0.6250
Epoch 29/100
1/1 [==============================] - 2s 2s/step - loss: 0.1436 - acc: 0.8438 - val_loss: 0.3132 - val_acc: 0.6250
Epoch 30/100
1/1 [==============================] - 2s 2s/step - loss: 0.3335 - acc: 0.4500 - val_loss: 0.2839 - val_acc: 0.3750
Epoch 31/100
1/1 [==============================] - 2s 2s/step - loss: 0.1816 - acc: 0.7750 - val_loss: 0.2856 - val_acc: 0.3750
Epoch 32/100
1/1 [==============================] - 2s 2s/step - loss: 0.1683 - acc: 0.7812 - val_loss: 0.2335 - val_acc: 0.6250
Epoch 33/100
1/1 [==============================] - 2s 2s/step - loss: 0.1989 - acc: 0.6500 - val_loss: 0.2703 - val_acc: 0.3750
Epoch 34/100
1/1 [==============================] - 2s 2s/step - loss: 0.1671 - acc: 0.7500 - val_loss: 0.3322 - val_acc: 0.3750
Epoch 35/100
1/1 [==============================] - 1s 1s/step - loss: 0.1560 - acc: 0.7500 - val

Epoch 92/100
1/1 [==============================] - 1s 1s/step - loss: 0.1519 - acc: 0.7812 - val_loss: 0.3612 - val_acc: 0.3750
Epoch 93/100
1/1 [==============================] - 1s 1s/step - loss: 0.1066 - acc: 0.8438 - val_loss: 0.3610 - val_acc: 0.3750
Epoch 94/100
1/1 [==============================] - 2s 2s/step - loss: 0.1607 - acc: 0.7250 - val_loss: 0.3066 - val_acc: 0.5000
Epoch 95/100
1/1 [==============================] - 2s 2s/step - loss: 0.1430 - acc: 0.8000 - val_loss: 0.3646 - val_acc: 0.3750
Epoch 96/100
1/1 [==============================] - 1s 1s/step - loss: 0.1384 - acc: 0.8438 - val_loss: 0.3627 - val_acc: 0.3750
Epoch 97/100
1/1 [==============================] - 1s 1s/step - loss: 0.0973 - acc: 0.8750 - val_loss: 0.4509 - val_acc: 0.5000
Epoch 98/100
1/1 [==============================] - 2s 2s/step - loss: 0.1726 - acc: 0.7500 - val_loss: 0.4348 - val_acc: 0.3750
Epoch 99/100
1/1 [==============================] - 1s 1s/step - loss: 0.1403 - acc: 0.7812 - val

Epoch 55/100
1/1 [==============================] - 1s 1s/step - loss: 0.1511 - acc: 0.7812 - val_loss: 0.2790 - val_acc: 0.5000
Epoch 56/100
1/1 [==============================] - 2s 2s/step - loss: 0.2352 - acc: 0.6750 - val_loss: 0.2047 - val_acc: 0.7500
Epoch 57/100
1/1 [==============================] - 1s 1s/step - loss: 0.1492 - acc: 0.8438 - val_loss: 0.2484 - val_acc: 0.6250
Epoch 58/100
1/1 [==============================] - 2s 2s/step - loss: 0.1352 - acc: 0.8000 - val_loss: 0.2050 - val_acc: 0.7500
Epoch 59/100
1/1 [==============================] - 2s 2s/step - loss: 0.1293 - acc: 0.8500 - val_loss: 0.1844 - val_acc: 0.7500
Epoch 60/100
1/1 [==============================] - 2s 2s/step - loss: 0.1480 - acc: 0.8438 - val_loss: 0.2665 - val_acc: 0.5000
Epoch 61/100
1/1 [==============================] - 1s 1s/step - loss: 0.1540 - acc: 0.7812 - val_loss: 0.2054 - val_acc: 0.6250
Epoch 62/100
1/1 [==============================] - 2s 2s/step - loss: 0.1586 - acc: 0.7500 - val

...

Epoch 99/100
1/1 [==============================] - 2s 2s/step - loss: 0.0034 - acc: 1.0000 - val_loss: 0.1582 - val_acc: 0.7500
Epoch 100/100
8/8 [==============================] - 0s 17ms/step
[0.31392136216163635, 0.625]

Fold  7
Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.1301 - acc: 0.8750 - val_loss: 0.0507 - val_acc: 0.8750
Epoch 2/100
1/1 [==============================] - 3s 3s/step - loss: 0.0539 - acc: 0.9500 - val_loss: 0.0033 - val_acc: 1.0000
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 0.1266 - acc: 0.8438 - val_loss: 0.1247 - val_acc: 0.8750
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 0.1296 - acc: 0.8250 - val_loss: 0.0287 - val_acc: 1.0000
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.0243 - acc: 0.9688 - val_loss: 1.7704e-04 - val_acc: 1.0000
Epoch 6/100
1/1 [==============================] - 2s 2s/step - loss: 0.0619 - acc: 0.9250 - val_loss: 0.0104 - val_acc: 1.

Epoch 62/100
1/1 [==============================] - 2s 2s/step - loss: 0.0529 - acc: 0.9500 - val_loss: 0.0460 - val_acc: 0.8750
Epoch 63/100
1/1 [==============================] - 2s 2s/step - loss: 0.0306 - acc: 0.9500 - val_loss: 1.7661e-04 - val_acc: 1.0000
Epoch 64/100
1/1 [==============================] - 2s 2s/step - loss: 0.0629 - acc: 0.9062 - val_loss: 0.0853 - val_acc: 0.8750
Epoch 65/100
1/1 [==============================] - 1s 1s/step - loss: 0.1606 - acc: 0.7500 - val_loss: 0.1649 - val_acc: 0.7500
Epoch 66/100
1/1 [==============================] - 2s 2s/step - loss: 0.1061 - acc: 0.8750 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 67/100
1/1 [==============================] - 1s 1s/step - loss: 0.0273 - acc: 0.9688 - val_loss: 0.0640 - val_acc: 0.8750
Epoch 68/100
1/1 [==============================] - 2s 2s/step - loss: 0.0330 - acc: 0.9750 - val_loss: 0.0527 - val_acc: 0.8750
Epoch 69/100
1/1 [==============================] - 2s 2s/step - loss: 0.0358 - acc: 0.9750 -

Epoch 25/100
1/1 [==============================] - 2s 2s/step - loss: 0.0454 - acc: 0.9500 - val_loss: 6.4531e-04 - val_acc: 1.0000
Epoch 26/100
1/1 [==============================] - 2s 2s/step - loss: 0.0534 - acc: 0.9062 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 27/100
1/1 [==============================] - 1s 1s/step - loss: 0.0590 - acc: 0.9375 - val_loss: 0.0010 - val_acc: 1.0000
Epoch 28/100
1/1 [==============================] - 2s 2s/step - loss: 0.0193 - acc: 0.9750 - val_loss: 1.8009e-04 - val_acc: 1.0000
Epoch 29/100
1/1 [==============================] - 1s 1s/step - loss: 0.0164 - acc: 0.9688 - val_loss: 1.8520e-04 - val_acc: 1.0000
Epoch 30/100
1/1 [==============================] - 2s 2s/step - loss: 0.0427 - acc: 0.9500 - val_loss: 0.2286 - val_acc: 0.7500
Epoch 31/100
1/1 [==============================] - 2s 2s/step - loss: 0.1584 - acc: 0.8000 - val_loss: 0.1750 - val_acc: 0.7500
Epoch 32/100
1/1 [==============================] - 2s 2s/step - loss: 0.1069 - acc: 

Epoch 89/100
1/1 [==============================] - 3s 3s/step - loss: 0.0115 - acc: 0.9750 - val_loss: 0.0131 - val_acc: 1.0000
Epoch 90/100
1/1 [==============================] - 2s 2s/step - loss: 0.0290 - acc: 0.9688 - val_loss: 0.0387 - val_acc: 0.8750
Epoch 91/100
1/1 [==============================] - 2s 2s/step - loss: 0.0261 - acc: 0.9750 - val_loss: 0.0290 - val_acc: 1.0000
Epoch 92/100
1/1 [==============================] - 2s 2s/step - loss: 0.0182 - acc: 1.0000 - val_loss: 0.1759 - val_acc: 0.7500
Epoch 93/100
1/1 [==============================] - 2s 2s/step - loss: 0.0546 - acc: 0.9250 - val_loss: 0.1742 - val_acc: 0.7500
Epoch 94/100
1/1 [==============================] - 2s 2s/step - loss: 0.0731 - acc: 0.8750 - val_loss: 0.4274 - val_acc: 0.5000
Epoch 95/100
1/1 [==============================] - 2s 2s/step - loss: 0.1202 - acc: 0.8438 - val_loss: 0.2556 - val_acc: 0.6250
Epoch 96/100
1/1 [==============================] - 3s 3s/step - loss: 0.1611 - acc: 0.8000 - val

Epoch 52/100
1/1 [==============================] - 2s 2s/step - loss: 0.0476 - acc: 0.9688 - val_loss: 0.1214 - val_acc: 0.8750
Epoch 53/100
1/1 [==============================] - 2s 2s/step - loss: 0.0403 - acc: 0.9750 - val_loss: 0.0066 - val_acc: 1.0000
Epoch 54/100
1/1 [==============================] - 2s 2s/step - loss: 0.0407 - acc: 0.9375 - val_loss: 2.7649e-04 - val_acc: 1.0000
Epoch 55/100
1/1 [==============================] - 2s 2s/step - loss: 0.0277 - acc: 0.9750 - val_loss: 6.2116e-04 - val_acc: 1.0000
Epoch 56/100
1/1 [==============================] - 2s 2s/step - loss: 0.0472 - acc: 0.9375 - val_loss: 5.0477e-04 - val_acc: 1.0000
Epoch 57/100
1/1 [==============================] - 2s 2s/step - loss: 0.0853 - acc: 0.8500 - val_loss: 6.6173e-04 - val_acc: 1.0000
Epoch 58/100
1/1 [==============================] - 2s 2s/step - loss: 0.0377 - acc: 0.9688 - val_loss: 5.4512e-04 - val_acc: 1.0000
Epoch 59/100
1/1 [==============================] - 2s 2s/step - loss: 0.0040

# Predicting

Define prediction function, takes in a model and a list of returned data from getFish function. Returns the model's prediction, the correct classification, the prediction weights, and the fish family/species.

In [10]:
def predict(model, get_fish_output):
    
    img, correct_class, fam = get_fish_output
    
    prediction_weights = model.predict(img)
    prediction = model.predict_classes(img)
    if (prediction[0] == 0):
        prediction = "eyespot"
    else:
        prediction = "noeyespot"

    #print "Model classification:",prediction,"\nCorrect classification:", correct_class,"\nPrediction wieghts:",prediction_weights,"\nType:",fam,"\n"
    return(prediction, correct_class, prediction_weights, fam)

# Writing the model's classification data to CSV

Loads model in "mod"  
Writes the model classification, correct classification, prediction weights, and species/family to a csv files  
Calculates data for confusion matrix

In [8]:
mod = load_model("second_try.h5")

with open('classification_data.csv', 'wb') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Model Classification', 'Correct Classification', 'Prediction Weights', 'Type'])

    true_eye=0
    true_noeye=0
    false_eye=0
    false_noeye=0

    for i in range(len(fish_paths)):
        info = predict(mod, getFish(fish_paths[i]))
        filewriter.writerow([info[0], info[1], info[2], info[3]])

        #computing info for confusion matrix
        if (info[0] == info[1]):
            if (info[0] == 'eyespot'):
                true_eye += 1
            else:
                true_noeye += 1
        else:
            if (info[0] == 'eyespot'):
                false_eye += 1
            else:
                false_noeye += 1

# Prints confusion matrix

In [9]:
data = {'Eyespots':[true_eye, false_noeye], 'No Eyespots':[false_eye, true_noeye] }
confusion_matrix = pd.DataFrame(data = data)
confusion_matrix = confusion_matrix.rename({0:'Eyespots (Model Output)', 1:'No Eyespots (Model Output)'},axis='index')
confusion_matrix

,Eyespots,No Eyespots
Eyespots (Model Output),38,1
No Eyespots (Model Output),2,39


# Running classifier on Chaets

Right now, the output in "info" is a little messy, but you can see the prediction and prediction weights. The function will automatically assume that "eyespot" is the correct classification due to how the function is written. It will also not list what the species name is. It's pretty messy for now, but I'll work on cleaning it up.

In [15]:
chaet_path = "/Users/leannwoo/Dropbox/machine_learning_fishes/randall/Chaetodontidae/"

chaets = []

for (dirpath, dirnames, filenames) in walk(chaet_path):
    for i in range(len(filenames)):
        img = dirpath + filenames[i]
        if "Store" not in img:  
            chaets.append(img)
    break
    
mod = load_model("second_try.h5")
info = []

for i in range(len(chaets)):
    info.append(predict(mod, getFish(chaets[i])))
    
info

[('noeyespot',
  'eyespot',
  array([[1.5743046e-06, 9.9999642e-01]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[5.3404574e-04, 9.9945408e-01]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[0.00922593, 0.99053675]], dtype=float32),
  ''),
 ('eyespot', 'eyespot', array([[0.6216251 , 0.54611564]], dtype=float32), ''),
 ('eyespot', 'eyespot', array([[0.96949995, 0.03259671]], dtype=float32), ''),
 ('noeyespot',
  'eyespot',
  array([[6.2341300e-05, 9.9996173e-01]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[0.00692909, 0.9931799 ]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[4.5743766e-07, 9.9999857e-01]], dtype=float32),
  ''),
 ('noeyespot',
  'eyespot',
  array([[8.772105e-07, 9.999994e-01]], dtype=float32),
  ''),
 ('eyespot',
  'eyespot',
  array([[9.9934715e-01, 8.1560889e-04]], dtype=float32),
  ''),
 ('eyespot', 'eyespot', array([[0.64428985, 0.3350438 ]], dtype=float32), ''),
 ('noeyespot',
  'eyespot',
  array